<a href="https://colab.research.google.com/github/FadzararAI/Data_Checker/blob/main/Data_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlopen
import json
import random
import pandas

import pandas
import re

# User Data Generator using a free API

In [ ]:
#Initiate dictionary to save the data
person_list = {}

In [ ]:
def d_generator(num):
    def id_generator(): # Generates 16 digits of user ID randomly
        a = [str(random.randint(1,9)) for i in range(16)]
        b = ''.join(a)
        return b
    def person_generator(): # Using free API to generate user Data
        for i in range(num):
            response = urlopen("https://randomuser.me/api/")
            u_data = json.loads(response.read())
            person_list[u_data['results'][0]['name']['first']] = [u_data['results'][0]['name']['last'],u_data['results'][0]['location']['city'],id_generator()]
    person_generator()

In [ ]:
# Generates 100 people data
d_generator(100)

In [ ]:
# Seeing the rows of the data
person_list

{'Mstibog': ['Kolomackiy', 'Kerch', '4277286289684439'],
 'Gertrude': ['Bourgeois', 'Collonges', '4884536266993548'],
 'Domingo': ['Bravo', 'Elche', '9261543811457368'],
 'Brandon': ['Marshall', 'Fontana', '7115751848875236'],
 'Rozaliya': ['Gerashchenko', 'Noviy Bug', '4979856129886448'],
 'Sara': ['Andersen', 'København Ø', '4746178182344283'],
 'Øystein': ['Fosser', 'Geilo', '8632919489963314'],
 'Eleanor': ['Miller', 'Mullingar', '8182448729831742'],
 'Wieslaw': ['Knoll', 'Calbe (Saale)', '7698635646123669'],
 'Albert': ['Black', 'Youghal', '3322441458967761'],
 'Sharron': ['Marshall', 'Naas', '1825139565897978'],
 'Mathis': ['Clark', 'Victoria', '3572533172729751'],
 'Leo': ['Rintala', 'Toivakka', '3585165988366373'],
 'Amber': ['West', 'Londonderry', '2262322668982588'],
 'Aleksi': ['Seppanen', 'Teuva', '1844729811176676'],
 'Sander': ['Thomsen', 'København N', '6369672322169261'],
 'Erin': ['Clark', 'Darwin', '8957584246321155'],
 'Krish': ['Nayak', 'Giridih', '9666464986581569'

## Data splitting and Transforming

In [ ]:
# Traditionally splitting data
Name = []
City = []
ID = []

In [ ]:
for i,j in person_list.items():
    Name.append(i)
    City.append(j[1])
    ID.append(j[2])

In [ ]:
new_person_list = {"Name":[i for i in Name],"ID":[i for i in ID], "City":[i for i in City]}

In [ ]:
data = pandas.DataFrame(new_person_list)

In [ ]:
data.head()

,Name,ID,City
0,Mstibog,4277286289684439,Kerch
1,Gertrude,4884536266993548,Collonges
2,Domingo,9261543811457368,Elche
3,Brandon,7115751848875236,Fontana
4,Rozaliya,4979856129886448,Noviy Bug


# Abnormality Checker
This is quite useful to check digits and format errors in KTP.
Just need to change the code by a little.

In [ ]:
def abnormality_check(dataset,col):
    global abnormality # Initialize variable to be used globally
    abnormality = {}
    validity = 1 # Validation checking
    index = 0 # Rows measurement
    x,x2 = 0,0 # Just to differentiate
    for i in dataset[col]:
        if len(i) != 16:
            validity = 0
            x+=1
            abnormality["Invalid_lenght"+str(x)] = [i]
            abnormality["IL_row"+str(x)] = index
        if re.search("[A-Z]|[a-z]",i) != None:
            validity = 0
            x2+=1
            abnormality["Invalid_character"+str(x2)] = [i]
            abnormality["IC_row"+str(x2)] = index
        index += 1
    if validity == 0:
        print("Data Abnormality found!")
    else:
        print("Data is normal.")

In [ ]:
# Using the column "ID" to check if there are abnormalities
abnormality_check(data,"ID")

Data is normal.


# Spice up the data to see if the algorithm works as intended

In [ ]:
# Selection rows 38, 49, 52, 56, 65 and 68
to_rpl = [38, 49, 52, 56, 65, 68]

In [ ]:
for i in to_rpl:
  print(data["ID"][i])

2312449452767736
6828456966964723
2111452859854322
4245916929456224
3779596224433532
4789611959713865


In [ ]:
data["ID"] = data["ID"].replace(to_replace=['2312449452767736'],value='232449452767736') # Omit 1
data["ID"] = data["ID"].replace(to_replace=['6828456966964723'],value='682845696696423') # Omit 7
data["ID"] = data["ID"].replace(to_replace=['2111452859854322'],value='211145285985322') # Omit 4
data["ID"] = data["ID"].replace(to_replace=['3779596224433532'],value='3779S96224433S32') # 55 to SS
data["ID"] = data["ID"].replace(to_replace=['4789611959713865'],value='47896II959713865') # 11 to II

## Checking again using the algorithm

In [ ]:
abnormality_check(data,"ID")

Data Abnormality found!


In [ ]:
abnormality

{'Invalid_lenght1': ['232449452767736'],
 'IL_row1': 38,
 'Invalid_lenght2': ['682845696696423'],
 'IL_row2': 49,
 'Invalid_lenght3': ['211145285985322'],
 'IL_row3': 52,
 'Invalid_character1': ['3779S96224433S32'],
 'IC_row1': 65,
 'Invalid_character2': ['47896II959713865'],
 'IC_row2': 68}